In [16]:
import warnings
import pandas as pd

import inspect
import itertools

import numpy as np
from scipy import linalg
from numpy.linalg import inv
from numpy.linalg import pinv
from numpy.linalg import det

from matplotlib import pyplot as plt

from scipy.optimize import minimize

import time

warnings.filterwarnings('ignore')

def array1d(X, dtype=None, order=None):

    return np.asarray(np.atleast_1d(X), dtype=dtype, order=order)


def array2d(X, dtype=None, order=None):

    return np.asarray(np.atleast_2d(X), dtype=dtype, order=order)


def log_multivariate_normal_density(X, means, covars, min_covar=1.e-7):

    if hasattr(linalg, 'solve_triangular'):
        # only in scipy since 0.9
        solve_triangular = linalg.solve_triangular
    else:
        # slower, but works
        solve_triangular = linalg.solve
    n_samples, n_dim = X.shape
    nmix = len(means)
    log_prob = np.empty((n_samples, nmix))
    for c, (mu, cv) in enumerate(zip(means, covars)):
        try:
            cv_chol = linalg.cholesky(cv, lower=True)
        except linalg.LinAlgError:
            # The model is most probabily stuck in a component with too
            # few observations, we need to reinitialize this components
            cv_chol = linalg.cholesky(cv + min_covar * np.eye(n_dim),
                                      lower=True)
        cv_log_det = 2 * np.sum(np.log(np.diagonal(cv_chol)))
        try:
            cv_sol = solve_triangular(cv_chol, (X - mu).T, lower=True).T
        except ValueError:
            cv_sol = np.linalg.solve(cv_chol, (X - mu).T).T

        log_prob[:, c] = - .5 * (np.sum(cv_sol ** 2, axis=1) + \
                                     n_dim * np.log(2 * np.pi) + cv_log_det)

    return log_prob

def get_params(obj):
    try:
        # get names of every variable in the argument
        args = inspect.getargspec(obj.__init__)[0]
        args.pop(0)   # remove "self"

        # get values for each of the above in the object
        argdict = dict([(arg, obj.__getattribute__(arg)) for arg in args])
        return argdict
    except:
        raise ValueError("object has no __init__ method")


def preprocess_arguments(argsets, converters):

    result = {}
    for argset in argsets:
        for (argname, argval) in argset.items():
            # check that this argument is necessary
            if not argname in converters:
                raise ValueError("Unrecognized argument: {0}".format(argname))

            # potentially use this argument
            if argname not in result and argval is not None:
                # convert to right type
                argval = converters[argname](argval)

                # save
                result[argname] = argval

    # check that all arguments are covered
    if not len(converters.keys()) == len(result.keys()):
        missing = set(converters.keys()) - set(result.keys())
        s = "The following arguments are missing: {0}".format(list(missing))
        raise ValueError(s)

    return result


def check_random_state(seed):
    """Turn seed into a np.random.RandomState instance

    If seed is None, return the RandomState singleton used by np.random.
    If seed is an int, return a new RandomState instance seeded with seed.
    If seed is already a RandomState instance, return it.
    Otherwise raise ValueError.
    """
    if seed is None or seed is np.random:
        return np.random.mtrand._rand
    if isinstance(seed, (int, np.integer)):
        return np.random.RandomState(seed)
    if isinstance(seed, np.random.RandomState):
        return seed
    raise ValueError('{0} cannot be used to seed a numpy.random.RandomState'
                     + ' instance').format(seed)

# Dimensionality of each Kalman Filter parameter for a single time step
DIM = {
    'transition_matrices': 2,
    'transition_offsets': 1,
    'observation_matrices': 2,
    'observation_offsets': 1,
    'transition_covariance': 2,
    'observation_covariance': 2,
    'initial_state_mean': 1,
    'initial_state_covariance': 2,
    'r':1,
    'g':1,
    'sigma_t':1
}


def _arg_or_default(arg, default, dim, name):
    if arg is None:
        result = np.asarray(default)
    else:
        result = arg
    if len(result.shape) > dim:
        raise ValueError(
            ('%s is not constant for all time.'
             + '  You must specify it manually.') % (name,)
        )
    return result


def _determine_dimensionality(variables, default):

    # gather possible values based on the variables
    candidates = []
    for (v, converter, idx) in variables:
        if v is not None:
            v = converter(v)
            candidates.append(v.shape[idx])

    # also use the manually specified default
    if default is not None:
        candidates.append(default)

    # ensure consistency of all derived values
    if len(candidates) == 0:
        return 1
    else:
        if not np.all(np.array(candidates) == candidates[0]):
            raise ValueError(
                "The shape of all " +
                "parameters is not consistent.  " +
                "Please re-check their values."
            )
        return candidates[0]


def _last_dims(X, t, ndims=2):

    X = np.asarray(X)
    if len(X.shape) == ndims + 1:
        return X[t]
    elif len(X.shape) == ndims:
        return X
    else:
        raise ValueError(("X only has %d dimensions when %d" +
                " or more are required") % (len(X.shape), ndims))


def _loglikelihoods(observation_matrices, observation_offsets,
                    observation_covariance, predicted_state_means,
                    predicted_state_covariances, observations,f,psi):

    n_timesteps = observations.shape[0]
    loglikelihoods = np.zeros(n_timesteps)
    for t in range(n_timesteps):
        observation = observations[t]
        if not np.any(np.ma.getmask(observation)):
            observation_matrix = _last_dims(observation_matrices, t)*f[t]
            observation_offset = _last_dims(observation_offsets, t, ndims=1)*psi[t]
            predicted_state_mean = _last_dims(
                predicted_state_means, t, ndims=1
            )
            predicted_state_covariance = _last_dims(
                predicted_state_covariances, t
            )

            predicted_observation_mean = (
                np.dot(observation_matrix,
                       predicted_state_mean)
                + observation_offset
            )
            predicted_observation_covariance = (
                np.dot(observation_matrix,
                       np.dot(predicted_state_covariance,
                              observation_matrix.T))
                + observation_covariance
            )
            loglikelihoods[t] = log_multivariate_normal_density(
                observation[np.newaxis, :],
                predicted_observation_mean[np.newaxis, :],
                predicted_observation_covariance[np.newaxis, :, :]
            )
    return loglikelihoods


def _loglikelihoods2(observation_matrices, observation_offsets,
                    observation_covariance, predicted_state_means,
                    predicted_state_covariances, observations,f,psi):

    n_timesteps = observations.shape[0]
    loglikelihoods = np.zeros(n_timesteps)
    for t in range(n_timesteps):
        observation = observations[t]
        if not np.any(np.ma.getmask(observation)):
            observation_matrix = _last_dims(observation_matrices, t)*f[t]
            observation_offset = _last_dims(observation_offsets, t, ndims=1)*psi[t]
            predicted_state_mean = _last_dims(
                predicted_state_means, t, ndims=1
            )
            predicted_state_covariance = _last_dims(
                predicted_state_covariances, t
            )

            predicted_observation_mean = (
                np.dot(observation_matrix,
                       predicted_state_mean)
                + observation_offset
            )
            predicted_observation_covariance = (
                np.dot(observation_matrix,
                       np.dot(predicted_state_covariance,
                              observation_matrix.T))
                + observation_covariance
            )
            loglikelihoods[t] = gauss_pdf(
                observation[np.newaxis, :],
                predicted_observation_mean[np.newaxis, :],
                predicted_observation_covariance[np.newaxis, :, :]
            )
    return loglikelihoods



def _filter_predict(transition_matrix, transition_covariance,
                    transition_offset, current_state_mean,
                    current_state_covariance):

    predicted_state_mean = (
        np.dot(transition_matrix, current_state_mean)
        + transition_offset
    )
    predicted_state_covariance = (
        np.dot(transition_matrix,
               np.dot(current_state_covariance,
                      transition_matrix.T))
        + transition_covariance
    )

    return (predicted_state_mean, predicted_state_covariance)


def _filter_correct(observation_matrix, observation_covariance,
                    observation_offset, predicted_state_mean,
                    predicted_state_covariance, observation):

    if not np.any(np.ma.getmask(observation)):
        predicted_observation_mean = (
            np.dot(observation_matrix,
                   predicted_state_mean)
            + observation_offset
        )
        predicted_observation_covariance = (
            np.dot(observation_matrix,
                   np.dot(predicted_state_covariance,
                          observation_matrix.T))
            + observation_covariance
        )

        kalman_gain = (
            np.dot(predicted_state_covariance,
                   np.dot(observation_matrix.T,
                          linalg.pinv(predicted_observation_covariance)))
        )

        corrected_state_mean = (
            predicted_state_mean
            + np.dot(kalman_gain, observation - predicted_observation_mean)
        )
        corrected_state_covariance = (
            predicted_state_covariance
            - np.dot(kalman_gain,
                     np.dot(observation_matrix,
                            predicted_state_covariance))
        )
    else:
        n_dim_state = predicted_state_covariance.shape[0]
        n_dim_obs = observation_matrix.shape[0]
        kalman_gain = np.zeros((n_dim_state, n_dim_obs))

        corrected_state_mean = predicted_state_mean
        corrected_state_covariance = predicted_state_covariance

    return (kalman_gain, corrected_state_mean,
            corrected_state_covariance)


def _filter(transition_matrices, observation_matrices, transition_covariance,
            observation_covariance, transition_offsets, observation_offsets,
            initial_state_mean, initial_state_covariance, observations,r,g,sigma_t):

    n_timesteps = observations.shape[0]
    n_dim_state = len(initial_state_mean)
    n_dim_obs = observations.shape[1]
    
    alpha = np.zeros((n_timesteps))
    psi = np.zeros((n_timesteps))
    f = np.zeros((n_timesteps))

    predicted_state_means = np.zeros((n_timesteps, n_dim_state))
    predicted_state_covariances = np.zeros(
        (n_timesteps, n_dim_state, n_dim_state)
    )
    kalman_gains = np.zeros((n_timesteps, n_dim_state, n_dim_obs))
    filtered_state_means = np.zeros((n_timesteps, n_dim_state))
    filtered_state_covariances = np.zeros(
        (n_timesteps, n_dim_state, n_dim_state)
    )

    for t in range(n_timesteps):
        if t == 0:
            predicted_state_means[t] = initial_state_mean
            predicted_state_covariances[t] = initial_state_covariance
        else:
            transition_matrix = _last_dims(transition_matrices, t - 1)
            transition_covariance = _last_dims(transition_covariance, t - 1)
            transition_offset = _last_dims(transition_offsets, t - 1, ndims=1)
            predicted_state_means[t], predicted_state_covariances[t] = (
                _filter_predict(
                    transition_matrix,
                    transition_covariance,
                    transition_offset,
                    filtered_state_means[t - 1],
                    filtered_state_covariances[t - 1]
                )
            )
        f[t] = r[t] - psi[t] * (g[t] - np.square(sigma_t)[t])
        observation_matrix = _last_dims(observation_matrices, t)*f[t]
        observation_covariance = _last_dims(observation_covariance, t)
        observation_offset = _last_dims(observation_offsets, t, ndims=1)*psi[t]
        (kalman_gains[t], filtered_state_means[t],
         filtered_state_covariances[t]) = (
            _filter_correct(observation_matrix,
                observation_covariance,
                observation_offset,
                predicted_state_means[t],
                predicted_state_covariances[t],
                observations[t]
            )
        )
        try:
            psi[t+1] = psi[t] + filtered_state_means[:,1][t]*f[t]
            alpha[t+1] = alpha[t] + filtered_state_means[:,0][t]*f[t]
        except:
            pass

    return (predicted_state_means, predicted_state_covariances,
            kalman_gains, filtered_state_means,
            filtered_state_covariances,f,psi,alpha)

def gauss_pdf(X, M, S):     

    DX = X-M           
    E = 0.5 * np.dot(DX.T, np.dot(inv(S), DX))         
    E = E + 0.5 * len(M) * np.log(2 * np.pi) + 0.5 * np.log(det(S))         

    return (-1*E)


class KalmanFilter(object):

    def __init__(self, transition_matrices=None, observation_matrices=None,
            transition_covariance=None, observation_covariance=None,
            transition_offsets=None, observation_offsets=None,
            initial_state_mean=None, initial_state_covariance=None,
            random_state=None,
            n_dim_state=None, n_dim_obs=None,r=None,g=None,sigma_t = None):
        """Initialize Kalman Filter"""

        # determine size of state space
        n_dim_state = _determine_dimensionality(
            [(transition_matrices, array2d, -2),
             (transition_offsets, array1d, -1),
             (transition_covariance, array2d, -2),
             (initial_state_mean, array1d, -1),
             (initial_state_covariance, array2d, -2),
             (observation_matrices, array2d, -1)],
            n_dim_state
        )
        n_dim_obs = _determine_dimensionality(
            [(observation_matrices, array2d, -2),
             (observation_offsets, array1d, -1),
             (observation_covariance, array2d, -2)],
            n_dim_obs
        )

        self.transition_matrices = transition_matrices
        self.observation_matrices = observation_matrices
        self.transition_covariance = transition_covariance
        self.observation_covariance = observation_covariance
        self.transition_offsets = transition_offsets
        self.observation_offsets = observation_offsets
        self.initial_state_mean = initial_state_mean
        self.initial_state_covariance = initial_state_covariance
        self.random_state = random_state
        self.n_dim_state = n_dim_state
        self.n_dim_obs = n_dim_obs
        self.r = r
        self.g = g
        self.sigma_t = sigma_t


    def filter(self, X):

        Z = self._parse_observations(X)

        (transition_matrices, transition_offsets, transition_covariance,
         observation_matrices, observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance,r,g,sigma_t) = (
            self._initialize_parameters()
        )

        (_, _, _, filtered_state_means,
         filtered_state_covariances,f,psi,alpha) = (
            _filter(
                transition_matrices, observation_matrices,
                transition_covariance, observation_covariance,
                transition_offsets, observation_offsets,
                initial_state_mean, initial_state_covariance,
                Z,r,g,sigma_t
            )
        )
        return (filtered_state_means, filtered_state_covariances,f,psi,alpha)

    def filter_update(self, filtered_state_mean, filtered_state_covariance,
                      observation=None, transition_matrix=None,
                      transition_offset=None, transition_covariance=None,
                      observation_matrix=None, observation_offset=None,
                      observation_covariance=None):

        # initialize matrices
        (transition_matrices, transition_offsets, transition_cov,
         observation_matrices, observation_offsets, observation_cov,
         initial_state_mean, initial_state_covariance,r,g,sigma_t) = (
            self._initialize_parameters()
        )
        transition_offset = _arg_or_default(
            transition_offset, transition_offsets,
            1, "transition_offset"
        )
        observation_offset = _arg_or_default(
            observation_offset, observation_offsets,
            1, "observation_offset"
        )
        transition_matrix = _arg_or_default(
            transition_matrix, transition_matrices,
            2, "transition_matrix"
        )
        observation_matrix = _arg_or_default(
            observation_matrix, observation_matrices,
            2, "observation_matrix"
        )
        transition_covariance = _arg_or_default(
            transition_covariance, transition_cov,
            2, "transition_covariance"
        )
        observation_covariance = _arg_or_default(
            observation_covariance, observation_cov,
            2, "observation_covariance"
        )

      
        if observation is None:
            n_dim_obs = observation_covariance.shape[0]
            observation = np.ma.array(np.zeros(n_dim_obs))
            observation.mask = True
        else:
            observation = np.ma.asarray(observation)

        predicted_state_mean, predicted_state_covariance = (
            _filter_predict(
                transition_matrix, transition_covariance,
                transition_offset, filtered_state_mean,
                filtered_state_covariance
            )
        )
        (_, next_filtered_state_mean,
         next_filtered_state_covariance) = (
            _filter_correct(
                observation_matrix, observation_covariance,
                observation_offset, predicted_state_mean,
                predicted_state_covariance, observation
            )
        )

        return (next_filtered_state_mean, next_filtered_state_covariance)



    def loglikelihood(self, X):

        Z = self._parse_observations(X)

        # initialize parameters
        (transition_matrices, transition_offsets,
         transition_covariance, observation_matrices,
         observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance,r,g,sigma_t) = (
            self._initialize_parameters()
        )

        # apply the Kalman Filter
        (predicted_state_means, predicted_state_covariances,
         kalman_gains, filtered_state_means,
         filtered_state_covariances,f,psi,alpha) = (
            _filter(
                transition_matrices, observation_matrices,
                transition_covariance, observation_covariance,
                transition_offsets, observation_offsets,
                initial_state_mean, initial_state_covariance,
                Z,r,g,sigma_t
            )
        )

        # get likelihoods for each time step
        loglikelihoods = _loglikelihoods(
          observation_matrices, observation_offsets, observation_covariance,
          predicted_state_means, predicted_state_covariances, Z,f,psi
        )

        return np.sum(loglikelihoods)
    
    def loglikelihood2(self, X):

        Z = self._parse_observations(X)

        # initialize parameters
        (transition_matrices, transition_offsets,
         transition_covariance, observation_matrices,
         observation_offsets, observation_covariance,
         initial_state_mean, initial_state_covariance,r,g,sigma_t) = (
            self._initialize_parameters()
        )

        # apply the Kalman Filter
        (predicted_state_means, predicted_state_covariances,
         kalman_gains, filtered_state_means,
         filtered_state_covariances,f,psi,alpha) = (
            _filter(
                transition_matrices, observation_matrices,
                transition_covariance, observation_covariance,
                transition_offsets, observation_offsets,
                initial_state_mean, initial_state_covariance,
                Z,r,g,sigma_t
            )
        )

        # get likelihoods for each time step
        loglikelihoods = _loglikelihoods2(
          observation_matrices, observation_offsets, observation_covariance,
          predicted_state_means, predicted_state_covariances, Z,f,psi
        )

        return np.sum(loglikelihoods)    
    
    

    def _initialize_parameters(self):
        """Retrieve parameters if they exist, else replace with defaults"""
        n_dim_state, n_dim_obs = self.n_dim_state, self.n_dim_obs

        arguments = get_params(self)
        defaults = {
            'transition_matrices': np.eye(n_dim_state),
            'transition_offsets': np.zeros(n_dim_state),
            'transition_covariance': np.eye(n_dim_state),
            'observation_matrices': np.eye(n_dim_obs, n_dim_state),
            'observation_offsets': np.zeros(n_dim_obs),
            'observation_covariance': np.eye(n_dim_obs),
            'initial_state_mean': np.zeros(n_dim_state),
            'initial_state_covariance': np.eye(n_dim_state),
            'random_state': 0,
            'r': np.zeros(n_dim_obs),
            'g': np.zeros(n_dim_obs),
            'sigma_t':np.zeros(n_dim_obs)
        }
        
        converters = {
            'transition_matrices': array2d,
            'transition_offsets': array1d,
            'transition_covariance': array2d,
            'observation_matrices': array2d,
            'observation_offsets': array1d,
            'observation_covariance': array2d,
            'initial_state_mean': array1d,
            'initial_state_covariance': array2d,
            'random_state': check_random_state,
            'n_dim_state': int,
            'n_dim_obs': int,
            'r':array1d,
            'g':array1d,
            'sigma_t':array1d
            
        }

        parameters = preprocess_arguments([arguments, defaults], converters)

        return (
            parameters['transition_matrices'],
            parameters['transition_offsets'],
            parameters['transition_covariance'],
            parameters['observation_matrices'],
            parameters['observation_offsets'],
            parameters['observation_covariance'],
            parameters['initial_state_mean'],
            parameters['initial_state_covariance'],
            parameters['r'],
            parameters['g'],
            parameters['sigma_t']
        )

    def _parse_observations(self, obs):
        """Safely convert observations to their expected format"""
        obs = np.ma.atleast_2d(obs)
        if obs.shape[0] == 1 and obs.shape[1] > 1:
            obs = obs.T
        return obs

def log_like(x):
    transition_covariance[0,0] = x[0]
    transition_covariance[1,1] = x[1]
    observation_covariance  = x[2]   
    
    return -1*kf.loglikelihood(observations) 

In [17]:
data = pd.read_stata('KalmanLearningData_HF(1).dta') ## add path here
data1 = data[['crsp_cl_grp','quarterly', 'STDvix_start', 'L1STDvix_start', 'L1WKalmanV', 'flows_idi',
       'L1STDvix']]
funds = list(data1['crsp_cl_grp'].unique())

data1

,crsp_cl_grp,quarterly,STDvix_start,L1STDvix_start,L1WKalmanV,flows_idi,L1STDvix
0,2000035.0,159.0,0.466807,0.115929,6.047455,-0.580481,0.387304
1,2000035.0,160.0,0.370077,0.466807,11.135160,1.055625,0.303128
2,2000035.0,161.0,0.837800,0.370077,19.632845,0.533858,0.365660
3,2000035.0,162.0,-0.020538,0.837800,8.682064,3.662717,0.591069
4,2000035.0,163.0,0.605933,-0.020538,19.152636,-12.582031,-0.140310
...,...,...,...,...,...,...,...
106895,2032515.0,234.0,-0.816804,-0.212408,-4.228628,-7.804037,-0.627252
106896,2032515.0,235.0,-0.084380,-0.816804,-4.053855,-11.045771,-0.942515
106897,2032515.0,236.0,-0.058386,-0.084380,3.363966,2.708376,0.099199
106898,2032515.0,237.0,-0.840244,-0.058386,-4.024742,3.454355,-0.483326


In [18]:
b = (0.001,10000) ## bounds on params
bnds = (b,b,b)


x0 =[0.1, 0.1 , 0.1 ] #starting params
params_names = ['sigma^2_alpha', 'sigma^2_beta' ,'R']


Master_Data = pd.DataFrame()
for i in range(2):
    data2 = data1[data1['crsp_cl_grp'] == funds[i]].reset_index(drop=True)
    data2 = data2[1:].reset_index(drop=True)

    r = data2['L1WKalmanV']
    g = data2['L1STDvix']
    sigma_t = data2['L1STDvix_start']
    sigma_t1 = data2['STDvix_start']
    q_t1 = data2['flows_idi']
    
    transition_matrix  =  [[0,0],[0,0]]
    observation_matrices = np.vstack( [np.ones(sigma_t1.shape),sigma_t1**2]).T[:, np.newaxis]
    transition_covariance = np.eye(2)
    observation_covariance = 1
    transition_offsets = [0,0]
    observation_offsets = np.array(sigma_t1**2 - sigma_t**2)[:, np.newaxis]
    initial_state_mean = [0,0]
    initial_state_covariance = [[1,0],[0,1]]
    observations = q_t1
    
    kf = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrices,
                  transition_covariance = transition_covariance,
                  observation_covariance = observation_covariance,
                 transition_offsets = transition_offsets, 
                  observation_offsets = observation_offsets, 
                  initial_state_mean = initial_state_mean, 
                  initial_state_covariance = initial_state_covariance,r=r,g=g,sigma_t = sigma_t)

    sol =  minimize(log_like, x0, method='L-BFGS-B', bounds = bnds)
    x =sol.x
    transition_covariance[0,0] = x[0]
    transition_covariance[1,1] = x[1]
    observation_covariance  = x[2] 

    results = kf.filter(observations)
    L_alpha = results[0][:,0]
    L_psi = results[0][:,1]
    psi = results[3]
    f = results[2]
    alpha = results[4]
    pred_qt_1 = L_alpha*f+(L_psi*f*sigma_t1**2)+(psi*(sigma_t1**2 -sigma_t**2))
    MSE = np.mean((q_t1 - pred_qt_1)**2)
    
    data2['lambda_alpha'] = L_alpha
    data2['lambda_psi'] =L_psi
    data2['psi'] = psi
    data2['alpha'] = alpha
    data2['MSE'] = MSE
    data2['sigma_alpha'] = x[0]
    data2['sigma_psi'] = x[1]
    
    
    Master_Data = Master_Data.append(data2)
    
Master_Data.to_stata('KalmanFilterData.dta')    ## add path here

Master_Data

,crsp_cl_grp,quarterly,STDvix_start,L1STDvix_start,L1WKalmanV,flows_idi,L1STDvix,lambda_alpha,lambda_psi,psi,alpha,MSE,sigma_alpha,sigma_psi
0,2000035.0,160.0,0.370077,0.466807,11.135160,1.055625,0.303128,0.092325,0.012645,0.000000,0.000000,0.434404,0.152549,0.021283
1,2000035.0,161.0,0.837800,0.370077,19.632845,0.533858,0.365660,0.021347,0.002090,0.140799,1.028051,0.434404,0.152549,0.021283
2,2000035.0,162.0,-0.020538,0.837800,8.682064,3.662717,0.591069,0.400849,0.000024,0.181773,1.446464,0.434404,0.152549,0.021283
3,2000035.0,163.0,0.605933,-0.020538,19.152636,-12.582031,-0.140310,-0.636232,-0.032590,0.181978,4.934739,0.434404,0.152549,0.021283
4,2000035.0,164.0,0.572711,0.605933,-13.272604,5.090257,0.729937,-0.367372,-0.016811,-0.443050,-7.267079,0.434404,0.152549,0.021283
5,2000035.0,165.0,0.952280,0.572711,-3.780593,-3.073272,0.693054,0.499411,0.063185,-0.222621,-2.450138,0.434404,0.152549,0.021283
6,2000035.0,166.0,0.265511,0.952280,-12.738756,-3.749109,0.463961,0.306975,0.003019,-0.456362,-4.297620,0.434404,0.152549,0.021283
7,2000035.0,167.0,1.493794,0.265511,0.211690,-1.016374,0.649067,0.003873,0.001206,-0.495434,-8.270149,0.434404,0.152549,0.021283
8,2000035.0,168.0,0.258034,1.493794,-0.226650,-1.693554,0.970556,0.323064,0.003001,-0.494833,-8.268219,0.434404,0.152549,0.021283
9,2000035.0,169.0,-0.019658,0.258034,16.649153,-3.051504,0.138239,-0.180612,-0.000010,-0.497385,-8.543007,0.434404,0.152549,0.021283
